In [1]:
import numpy as np
from pymsm.simulation import MultiStateSimulator, default_update_covariates_function

%load_ext autoreload
%autoreload 2

# Rossi - single state example

In [4]:
# Load rossi dataset
from lifelines.datasets import load_rossi

rossi = load_rossi()
from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(rossi, duration_col="week", event_col="arrest")
baseline_hazard = cph.baseline_hazard_["baseline hazard"]
coefs = cph.params_

from pymsm.datasets import load_rossi_competing_risk_data

rossi_competing_risk_data, covariate_names = load_rossi_competing_risk_data()

# Define the full model
competing_risks_models_list = [
    {
        "origin_state": 1,
        "target_states": [2],
        "model_defs": {"coefs": coefs, "baseline_hazard": baseline_hazard},
    }
]

# Configure the simulator
mssim = MultiStateSimulator(
    competing_risks_models_list,
    terminal_states=[2],
    update_covariates_fn=default_update_covariates_function,
    covariate_names=covariate_names,
)

# Run simulation
mc_paths = mssim.run_monte_carlo_simulation(
    sample_covariates=rossi_competing_risk_data.loc[0, covariate_names],
    origin_state=1,
    current_time=0,
    n_random_samples=10,
    max_transitions=10,
    print_paths=True,
)


100%|██████████| 10/10 [00:00<00:00, 1114.94it/s]

[1, 2]
[45.0]
[1, 2]
[11.0]
[1, 2]
[27.0]
[1, 2]
[23.0]
[1, 2]
[19.0]
[1, 2]
[25.0]
[1, 2]
[37.0]
[1, 2]
[49.0]
[1, 2]
[24.0]
[1, 2]
[48.0]
